<a href="https://colab.research.google.com/github/MaikarfiJesse/transfer_learning_assignment/blob/main/Transfer_model_Dates_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dates fruits Classification

This project is aimed to use pretrained models with a new dataset to classify different types of dates fruits from their images.

##Import libraries

In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, BinaryAccuracy, Precision, Recall, AUC
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import cv2
import pathlib
import matplotlib as mpl

##INSTALL DATASET


In [2]:
img_width = 448
img_height = 448
batch_size = 64
color = 3
dataset_folder = "date-fruit-image-dataset-in-controlled-environment"
dataset_url = 'https://www.kaggle.com/wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment'
classes_name = ['Ajwa', 'Galaxy', 'Medjool', 'Meneifi', 'Nabtat Ali', 'Rutab', 'Shaishe', 'Sokari', 'Sugaey']


In [3]:
!pip install opendatasets

import opendatasets as od
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jessemaikarfi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment


100%|██████████| 3.11G/3.11G [00:34<00:00, 97.7MB/s]


## Read the dataset

In [4]:
dataset = pathlib.Path(dataset_folder)

def directory_dataset(dataset):
  folders = []
  for i in dataset.iterdir():
    if i.is_dir():
      folders.append(i)
  return folders

folders = directory_dataset(dataset)

### Total number of Images

In [5]:
def number_of_images_in_dataset(dataset):
  images = list(dataset.glob("*/*.*"))
  return len(images)
print("number of all images in dataset: {}".format(number_of_images_in_dataset(dataset)))

number of all images in dataset: 1658


###number of Images per folder

In [6]:
def number_of_images_in_each_folder(folders):
  for i in folders:
    str_ = "{}: {}".format(i, len(list(pathlib.Path(i).glob("*.*"))))
    print(str_)

number_of_images_in_each_folder(folders)

date-fruit-image-dataset-in-controlled-environment/Galaxy: 190
date-fruit-image-dataset-in-controlled-environment/Rutab: 146
date-fruit-image-dataset-in-controlled-environment/Meneifi: 232
date-fruit-image-dataset-in-controlled-environment/Nabtat Ali: 177
date-fruit-image-dataset-in-controlled-environment/Medjool: 135
date-fruit-image-dataset-in-controlled-environment/Shaishe: 171
date-fruit-image-dataset-in-controlled-environment/Sokari: 264
date-fruit-image-dataset-in-controlled-environment/Sugaey: 168
date-fruit-image-dataset-in-controlled-environment/Ajwa: 175


###Blurring images

In [7]:
def Blurring(image):
  image = cv2.medianBlur(image,5)
  return image

def image_generator(dataset_folder):
  datagen = ImageDataGenerator( rescale=1/255, validation_split=0.1,vertical_flip=True ,horizontal_flip=True,width_shift_range=0.2,height_shift_range=0.2,
                               rotation_range = 5, shear_range = 0.02,zoom_range = 0.02, preprocessing_function = Blurring)

  train_generator = datagen.flow_from_directory(
      dataset_folder,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='categorical',
      shuffle=True,
      subset='training')

  validation_generator = datagen.flow_from_directory(
      dataset_folder,
      target_size=(img_height, img_width),
      batch_size= batch_size,
      shuffle=False,
      class_mode='categorical',
      subset='validation')

  return [train_generator, validation_generator]

In [8]:

[training_dataset, validation_dataset] = image_generator(dataset)

Found 1496 images belonging to 9 classes.
Found 162 images belonging to 9 classes.


##Pretrained Models

In [9]:
from tensorflow.keras.applications import VGG16, MobileNetV2, VGG19
from tensorflow.keras.models import Model


###Load pre-trained models


In [10]:
# Load pre-trained models
base_models = [VGG16, MobileNetV2, VGG19]
pre_models = []

for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False, input_shape=(img_height, img_width, color))
    pre_models.append(base)


for pretrained_model in pre_models:
    for layer in pretrained_model.layers:
        layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


80134624/80134624 [==============================] - 1s 0us/step


###layers modification

In [11]:
models = []
for pretrained_model in pre_models:
    last_layer = Flatten()(pretrained_model.output)
    final_layer = Dense(9, activation='softmax')(last_layer)

    model = Model(inputs=pretrained_model.input, outputs=final_layer)
    models.append(model)

###Metrics

In [12]:
me = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'),
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc')
]

###Compiling

In [13]:
for model in models:
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=me)

In [14]:
call_1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)
callbacks = [call_1]

##Training

In [ ]:
epochs=5
history = models[0].fit(training_dataset, epochs=epochs, callbacks = callbacks, batch_size = batch_size
)

Epoch 1/5
24/24 [==============================] - 3412s 141s/step - loss: 7.0478 - tp: 239.0000 - fp: 928.0000 - tn: 11040.0000 - fn: 1257.0000 - accuracy: 0.8377 - precision: 0.2048 - recall: 0.1598 - auc: 0.5735
Epoch 2/5
24/24 [==============================] - 3427s 142s/step - loss: 1.6495 - tp: 551.0000 - fp: 403.0000 - tn: 11565.0000 - fn: 945.0000 - accuracy: 0.8999 - precision: 0.5776 - recall: 0.3683 - auc: 0.8404
Epoch 3/5
 1/24 [>.............................] - ETA: 1:04:38 - loss: 1.1103 - tp: 29.0000 - fp: 8.0000 - tn: 504.0000 - fn: 35.0000 - accuracy: 0.9253 - precision: 0.7838 - recall: 0.4531 - auc: 0.9196

In [ ]:
epochs=3
history = models[1].fit(training_dataset, epochs=epochs, callbacks = callbacks, batch_size = batch_size
)

In [ ]:
epochs=3
history = models[2].fit(training_dataset, epochs=epochs, callbacks = callbacks, batch_size = batch_size
)

Epoch 1/3
24/24 [==============================] - 4839s 201s/step - loss: 11.8589 - tp: 1433.0000 - fp: 1345.0000 - tn: 22591.0000 - fn: 1559.0000 - accuracy: 0.8922 - precision: 0.5158 - recall: 0.4789 - auc: 0.7623
Epoch 2/3
24/24 [==============================] - 4859s 201s/step - loss: 2.7576 - tp: 490.0000 - fp: 531.0000 - tn: 11437.0000 - fn: 1006.0000 - accuracy: 0.8858 - precision: 0.4799 - recall: 0.3275 - auc: 0.7699
Epoch 3/3
24/24 [==============================] - 4785s 198s/step - loss: 1.2485 - tp: 665.0000 - fp: 308.0000 - tn: 11660.0000 - fn: 831.0000 - accuracy: 0.9154 - precision: 0.6835 - recall: 0.4445 - auc: 0.9025
